In [33]:
BOARD_DIRECTIONS = (
    (1,0),
    (1,-1),
    (0,-1),
    (-1,-1),
    (-1,0),
    (-1,1),
    (0,1),
    (1,1),
)

def is_in_board(board, x, y):
    board_width, board_height = get_board_dims(board)
    return 0 <= x < board_width and 0 <= y < board_height

def is_valid_move(board, player, x, y):
    if not is_in_board(board, x, y) or board[y][x] != 0:
        return None
    other_player = get_other_player(player)
    
    board[y][x] = player # Temporarily place tile at x,y
    tiles_to_flip = []
    global BOARD_DIRECTIONS
    for x_delta, y_delta in BOARD_DIRECTIONS:
        x_curr = x + x_delta
        y_curr = y + y_delta
        if not is_in_board(board, x_curr, y_curr):
            continue
        tile_buffer = []
        while board[y_curr][x_curr] == other_player:
            tile_buffer.append((x_curr, y_curr))
            x_curr += x_delta
            y_curr += y_delta
            if not is_in_board(board, x_curr, y_curr):
                break
        if is_in_board(board, x_curr, y_curr) and board[y_curr][x_curr] == player:
            tiles_to_flip.extend(tile_buffer)
    board[y][x] = 0 # Remove temporary tile at x,y
    if len(tiles_to_flip) == 0:
        return None
    return tiles_to_flip

def get_board_dims(board):
    # returns (cols, rows)
    return (len(board[0]), len(board))

def get_valid_moves(board, player):
    cols, rows = get_board_dims(board)
    moves_dict = {}
    for x in range(cols):
        for y in range(rows):
            if board[y][x] == 0:
                tiles_to_flip = is_valid_move(board, player, x, y)
                if tiles_to_flip != None:
                    moves_dict[(x,y)] = tiles_to_flip
    return moves_dict
    
def flip_tiles_on_board(board, player, tiles_to_flip):
    for x,y in tiles_to_flip:
        board[y][x] = player
        
def get_other_player(player):
    if player == 1:
        return 2
    return 1

def evaluate_board(board, player):
    cols, rows = get_board_dims(board)
    other_player = get_other_player(player)
    num_tiles_for_player = 0
    num_tiles_for_other_player = 0
    for x in range(cols):
        for y in range(rows):
            tile = board[y][x]
            if tile == player:
                num_tiles_for_player += 1
            if tile == other_player:
                num_tiles_for_other_player += 1
    return num_tiles_for_player - num_tiles_for_other_player

class Node(object):
    def __init__(self):
        self.children = []
        self.parent = None
        self.eval_val = None
        self.propagated_val = None
        self.move = None # (x, y, player)
        
def build_tree(node, board, player):
    moves_dict = get_valid_moves(board, player)
    for x,y in moves_dict.keys():
        board_copy = list(board)
        board_copy[y][x] = player # Place player's tile at x,y
        flip_tiles_on_board(board_copy, player, moves_dict[(x,y)])
        
        new_node = Node()
        new_node.parent = node
        new_node.eval_val = evaluate_board(board, player)
        new_node.move = (x, y, player)
        node.children.append(new_node)
        
        build_tree(new_node, board_copy, get_other_player(player))

In [42]:
test_board = [
    [0,0,0],
    [1,2,0],
    [2,1,0],
]
parent_node = Node()
build_tree(parent_node, test_board, player=1)

TypeError: build_tree() got an unexpected keyword argument 'depth_limit'

In [41]:
parent_node.children[0].children[0].children[0].children[0].move

(0, 0, 2)

In [36]:
import unittest

class ReversiTests(unittest.TestCase):
    
    def test_build_tree(self):
        test_board = [
            [0,0,0],
            [1,2,0],
            [2,1,0],
        ]
        parent_node = Node()
        build_tree(parent_node, test_board, 1)
        self.assertEqual(parent_node.children[0].eval_val, 3)
        self.assertEqual(parent_node.children[0].move, (1, 0, 1))
        self.assertEqual(parent_node.children[1].move, (2, 1, 1))
        self.assertEqual(parent_node.children[0].children[0].children[0].move, (2, 1, 1))
    
    def test_get_valid_moves(self):
        test_board = [
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,1,2,0],
            [0,0,2,1,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(get_valid_moves(test_board, 1), {(4, 2): [(3, 2)], (1, 3): [(2, 3)], (3, 1): [(3, 2)], (2, 4): [(2, 3)]})
        
    def test_is_valid_move(self):
        test_board = [
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,1,2,0],
            [0,0,2,1,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(is_valid_move(test_board, 1, 3, 1), [(3,2)])
        self.assertEqual(is_valid_move(test_board, 1, 0, 0), None)
        test_board_2 = [
            [0,0,1,0,1],
            [0,0,2,2,0],
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(is_valid_move(test_board_2, 1, 2, 2), [(3,1), (2,1)])
        self.assertEqual(is_valid_move(test_board_2, 1, 3, 0), None)
        test_board_3 = [
            [0,0,0],
            [1,2,0],
            [2,1,0],
        ]
        self.assertEqual(is_valid_move(test_board_3, 1, 1, 0), [(1,1)])

    def test_is_in_board(self):
        test_board = [[0]*5]*5
        self.assertEqual(is_in_board(test_board, 4, 4), True)
        self.assertEqual(is_in_board(test_board, 6, 4), False)
        self.assertEqual(is_in_board(test_board, 4, -1), False)

suite = unittest.TestLoader().loadTestsFromTestCase(ReversiTests)
unittest.TextTestRunner(verbosity=2).run(suite)

test_build_tree (__main__.ReversiTests) ... ok
test_get_valid_moves (__main__.ReversiTests) ... ok
test_is_in_board (__main__.ReversiTests) ... ok
test_is_valid_move (__main__.ReversiTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>